<a href="https://colab.research.google.com/github/Natg4/Wordle/blob/main/Specification_2604%20avec%20seconde%20spec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# pip install openpyxl

import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import pandas as pd
import requests
import numpy as np
from scipy import stats





In [4]:
!pip install -q xlrd
!git clone https://github.com/Jeremstar/ca-va-predire-l-inflation-severe

!ls données/

Cloning into 'ca-va-predire-l-inflation-severe'...
remote: Enumerating objects: 151, done.
remote: Counting objects: 100% (79/79), done.
remote: Compressing objects: 100% (60/60), done.
remote: Total 151 (delta 46), reused 40 (delta 18), pack-reused 72
Receiving objects: 100% (151/151), 33.65 MiB | 19.62 MiB/s, done.
Resolving deltas: 100% (72/72), done.
ls: cannot access 'données/': No such file or directory


In [7]:
OECD = pd.read_excel('/content/ca-va-predire-l-inflation-severe/données/OECD - short term economic indicators.xlsx')

IMF = pd.read_excel('/content/ca-va-predire-l-inflation-severe/données/IMF - monetary policy.xlsx')

agregated_data=pd.ExcelFile('/content/ca-va-predire-l-inflation-severe/données/FRBNY-SCE-Data.xlsx')
agregated_anticipation= df1=pd.read_excel(agregated_data, 'Inflation expectations')

agrant = agregated_anticipation[['Unnamed: 1','Unnamed: 2']].drop([0,1,2]).reset_index().rename(columns={'index':'date','Unnamed: 1':'short exp','Unnamed: 2':'mid exp'})

Enq13 = pd.read_excel('https://www.newyorkfed.org/medialibrary/interactives/sce/sce/downloads/data/frbny-sce-public-microdata-complete-13-16.xlsx')

Enq17 = pd.read_excel('https://www.newyorkfed.org/medialibrary/interactives/sce/sce/downloads/data/frbny-sce-public-microdata-complete-17-19.xlsx')

Latest=pd.ExcelFile('https://www.newyorkfed.org/medialibrary/Interactives/sce/sce/downloads/data/frbny-sce-public-microdata-latest.xlsx')

Enq20=pd.read_excel(Latest, 'Data')

# ça c'était pour le 'W' mais comme j'avais que le PIB j'ai laissé tombé pour cette fois
#pib = pd.read_excel(r'C:\Users\3e3gr\OneDrive\Documents\ENSAE\2A-S2\StatApp\PIBUS1.xlsx').drop(list(range(0,35))).rename(columns = {'Produit intérieur brut des États-Unis 1980-2021' : 'date','Unnamed: 1':'PIB'}) #en milliard de $ courant

#pib.set_index('date',inplace=True)

# u = pd.read_excel(r'C:\Users\3e3gr\OneDrive\Documents\ENSAE\2A-S2\StatApp\chomageUS.xlsx')


In [8]:


#Il faut faire 2-3 manips pour rendre les bases utilisables
new_columns = Enq13.iloc[0]
Enq13.columns = new_columns
Enq13 = Enq13.drop(0)

new_columns = Enq17.iloc[0]
Enq17.columns = new_columns
Enq17 = Enq17.drop(0)
new_columns = Enq20.iloc[0]
Enq20.columns = new_columns
Enq20 = Enq20.drop(0)
Enq17['date'] = pd.to_datetime(Enq17['date'].astype(str) + '01', format='%Y%m%d')
Enq13['date'] = pd.to_datetime(Enq13['date'].astype(str) + '01', format='%Y%m%d')
Enq20['date'] = pd.to_datetime(Enq20['date'].astype(str) + '01', format='%Y%m%d')
Base_finale=pd.concat([Enq13,Enq17,Enq20])
#Base_finale.to_excel('Base_finale.xlsx')


In [9]:
#Ici certaines choses ont été raccourcis mais rien de fondamental n'a changé

# I - premiere spécification


from sklearn.linear_model import LinearRegression
from statsmodels.regression.linear_model import OLS
import statsmodels.api as sm
AA=Base_finale
list(AA.columns)


educ=pd.get_dummies(AA['_EDU_CAT'])
age=pd.get_dummies(AA['_AGE_CAT'])
income=pd.get_dummies(AA['_HH_INC_CAT'])
numeracy=pd.get_dummies(AA['_NUM_CAT'])
np.sum(educ['High School'])


BC = pd.concat([AA, educ], axis=1).reindex(AA.index) #BC est pour Base complète. Ici, on complète la base avec des dummies
BC = pd.concat([BC, age], axis=1).reindex(BC.index)
BC = pd.concat([BC, income], axis=1).reindex(BC.index)
BC = pd.concat([BC, numeracy], axis=1).reindex(BC.index)

"""on code mtn pour le genre :"""

gender=pd.get_dummies(AA['Q33'])
gender.rename(columns={1: "femme"}, inplace=True)
BC = pd.concat([BC, gender['femme']], axis=1).reindex(BC.index)

"""c fait pour le genre.
Faisons le ménage dans les non-réponses et dans les outliers"""

BC=BC.dropna(subset=['Q8v2part2','Q9bv2part2']) #On supprime les personnes qui n'ont pas répondu aux questions sur les anticipations d'inflation.
"""Supprimons les outliers"""

test=BC[BC['Q8v2part2']<=20]
test=test[test['Q8v2part2']>=-20]
max(test['Q8v2part2'])

BC.rename(columns={'Q8v2part2':'inflation anticipée'},inplace=True)

model = OLS(BC['inflation anticipée'].astype('float'),BC['femme'].astype('float'))
results = model.fit()
su = results.summary()
print(su)

# Calcul de l'inflation :

inflation_US = OECD[(OECD['Country'] == "United States") & (OECD['Subject2'] == "Consumer prices: all items")]
inflation_US2 = pd.DataFrame({
    'day': pd.to_datetime(inflation_US['Time'], format='%Y-%m-%d'),
    'Inflation': inflation_US['Value'].astype(float)
})

inflation_US3 = pd.DataFrame({
    'date': inflation_US2['day'],
    'tx_evol_ann_pct': (inflation_US2['Inflation'] / inflation_US2['Inflation'].shift(12) - 1) * 100
})
test['inflation t']=0
df3 = pd.merge(test,inflation_US3,how='outer',left_on=['date'],right_on=['date'])
df3=df3[df3['Q1'].notnull()]
BF=df3

#DEFINITION D'un indicateur d'instabilité personnelle

"""
On va définir les variables qu'il serait pertinent d'inclure :

Q3 : proba changement de résidence dans les douze prochain mois
Q10 : 3 : chomeurs
Q13new: proba de changer d'emploi
Q25v2 : 3 : anticipe baisse de revenus
Q30new : proba de ne pas réussir à payer un prêt

"""



BF.loc[BF['Q25v2']==3, 'loss expectation'] = 1 #On binarise le fait d'anticiper une baisse de ses revenus


"""
Faisons quelques statistiques descriptives dessus :
"""
import statistics as stat

def weighted_average(dataframe, value, weight):
    val = dataframe[value]
    wt = dataframe[weight]
    return (val * wt).sum() / wt.sum()


len(BF['Q8v2part2'])
#

BF.rename(columns={'tx_evol_ann_pct':'taux inflation','Q8v2part2':'inflation anticipée'},inplace=True)

BF.dropna(subset=['weight'], inplace=True)
poids = BF['weight'].astype('float')
Y = BF['inflation anticipée'].astype('float')
#


#Alternative : on sépare les différents aspects de l'instabilité personnelle pour ne pas leur donner un poids arbitraire

#Allure proba de changement de résidence sous 12 mois
BF['Q3']
print("nb de valeurs manquantes dans Q3 : "+str(BF['Q3'].isnull().sum()))
#440



#Allure de l'indicatrice du chômage
BF['Q10_3']
print("nb de valeurs manquantes dans Q10_3 : "+str(BF['Q10_3'].isnull().sum()))
# nb de valeurs manquantes dans Q10_3 : 0
# le chômage est tjrs renseigné, c'est cool



#Allure proba changer d'emploi
BF['Q13new']
print("nb de valeurs manquantes dans Q13new : "+str(BF['Q13new'].isnull().sum()))
# nb de valeurs manquantes dans Q13new : 53054
# c'est une catastrophe absolue, quasiment la moitié des participants n'y ont pas répondu... il faudra sans doute faire une régression qui ne le prend pas en compte



#Allure anticipation baisse revenus
BF['Q25v2']
print("nb de valeurs manquantes dans Q25v2 : "+str(BF['Q25v2'].isnull().sum()))
# nb de valeurs manquantes dans Q25v2 : 60



#Allure proba ne pas réussir à rembourser un prêt
BF['Q30new']
print("nb de valeurs manquantes dans Q30new : "+str(BF['Q30new'].isnull().sum()))
# nb de valeurs manquantes dans Q30new : 226



BF


<ipython-input-9-fc510ee59773>:27: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  gender=pd.get_dummies(AA['Q33'])


                                 OLS Regression Results                                 
Dep. Variable:     inflation anticipée   R-squared (uncentered):                   0.003
Model:                             OLS   Adj. R-squared (uncentered):              0.003
Method:                  Least Squares   F-statistic:                              418.8
Date:                 Tue, 25 Apr 2023   Prob (F-statistic):                    5.95e-93
Time:                         12:12:50   Log-Likelihood:                     -7.5224e+05
No. Observations:               140880   AIC:                                  1.504e+06
Df Residuals:                   140879   BIC:                                  1.504e+06
Df Model:                            1                                                  
Covariance Type:             nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------

<ipython-input-9-fc510ee59773>:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  BF.loc[BF['Q25v2']==3, 'loss expectation'] = 1 #On binarise le fait d'anticiper une baisse de ses revenus
<ipython-input-9-fc510ee59773>:97: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  BF.rename(columns={'tx_evol_ann_pct':'taux inflation','Q8v2part2':'inflation anticipée'},inplace=True)


nb de valeurs manquantes dans Q3 : 440
nb de valeurs manquantes dans Q10_3 : 0
nb de valeurs manquantes dans Q13new : 53412
nb de valeurs manquantes dans Q25v2 : 58
nb de valeurs manquantes dans Q30new : 224


,date,userid,tenure,weight,Q1,Q2,Q3,Q4new,Q5new,Q6new,...,Under 40,50k to 100k,Over 100k,Under 50k,High,Low,femme,inflation t,taux inflation,loss expectation
0,2013-06-01,70000220,6,16.327999,3,3,50,20,10,15,...,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.754417,1.0
1,2013-06-01,70000224,7,0.228,4,4,25,10,25,75,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.754417,NaN
2,2013-06-01,70000234,6,4.066,4,3,3,9,20,20,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.754417,NaN
3,2013-06-01,70000238,6,3.035,3,3,0,10,5,70,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.754417,NaN
4,2013-06-01,70000239,7,1.947,2,3,10,20,20,50,...,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.754417,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130639,2022-06-01,75007148,1,0.714652,3,2,0,22,100,10,...,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,9.059758,NaN
130640,2022-06-01,75007149,1,1.373903,1,1,11,0,0,30,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,9.059758,NaN
130641,2022-06-01,75007151,1,1.235504,2,4,25,30,0,85,...,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,9.059758,NaN
130642,2022-06-01,75007154,1,0.725876,4,3,100,80,70,90,...,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,9.059758,NaN


In [10]:
#Ca je l'ai ravancé sinon ça bloqué

#obtention d'une colonne proba diminution indices boursiers
BF['Q6new_bis'] = 100-BF['Q6new']
BF['Q6new_bis'] = BF['Q6new_bis'].astype(float)

C'est à partir de là que commence les modifications

In [11]:
##Tentative de maxi regression


#On renomme les variables
BF.rename(columns={'Q10_3':'chômage','Q3': 'proba chgt résidence','Q4new':'proba hausse chomage','Q5new':'proba hausse taux intêret','Q6new_bis':'proba diminution indices boursiers','Q25v2':'anticipation baisse revenus','Q30new':'proba non-remboursement prêt'},inplace=True)

BF.rename(columns={'High': 'High_num','Over 60':'age > 60 ans','Under 40':'age < 40 ans','College':'Niveau licence','Over 100k':'revenus > $100k','Under 50k':'revenus < $50k'}, inplace=True)

#On squarise les deux variables qui nous semblait chelou : hausse taux car ça paraissait être une relation quadratique || chgt residence (qui ne semblait pas avoir d'effet linéairement) pck on sait jamais
BF['proba hausse taux intêret square'] = (BF['proba hausse taux intêret'] - 50)**2
BF['proba chgt résidence square'] = (BF['proba chgt résidence']-50)**2


#Fonction qui permet de créer les variables multipliées par l'inflation
def produitisation(df,reg,a='taux inflation'):
    for r in reg:
        s = r + ' * inflation'
        df[s]=df[a]*df[r]
    return [r+' * inflation' for r in reg]

#Crée les indicatrices temporelles
def timisation(df,a=2013,b=2022):
    for d in range(a,b+1):
        df['t'+str(d)]=(df['date']>=str(d)+'-01-01')*1 - (df['date']>=str(d+1)+'-01-01')*1

timisation(BF)


#Ca c'est pour si on a besoin de la variable PIB que j'avais préparé

#def PIBisation(df,a=2013,b=2022):
#    v = float(pib.loc[str(a)])*df['t'+str(a)]
#    for d in range(a+1,b+1):
#        v = v + float(pib.loc[str(d)])*df['t'+str(d)]
#    return v
#
#BF['PIB'] = PIBisation(BF)

Maintenant que BF contient toutes les varibles qui nous intéresse, il ne nous reste plus qu'à sélectionner les variables sur lesquels on veut régresser à chaque fois : (_Prod est le vecteur contenant les varibles de _ multipliées par l'inflation)

In [12]:


Classicreg = ['taux inflation']
Xreg = ['femme','age > 60 ans','age < 40 ans','Niveau licence','revenus > $100k','revenus < $50k','High_num']
XregProd = produitisation(BF,Xreg)

Times1reg = ['t'+str(y) for y in range(2013,2020)] #pour les regressions "av" : de 2013 à 2019
Times2reg = ['t'+str(y) for y in range(2020,2023)] #pour les regressions "ap" : de été 2020 à 2022

Macroreg = ['proba hausse chomage','proba hausse taux intêret','proba hausse taux intêret square','proba diminution indices boursiers']#des proba sur 100
MacroregProd = produitisation(BF,Macroreg)

# Microreg = ['chômage','anticipation baisse revenus','proba chgt résidence','proba chgt résidence square','proba non-remboursement prêt']#indicatrice et des proba sur 100
Microreg = ['chômage','anticipation baisse revenus','proba non-remboursement prêt']#indicatrice et des proba sur 100
MicroregProd = produitisation(BF,Microreg)


#Toujours pour le PIB

#Wreg = ['PIB']
#WregProd = produitisation(BF,Wreg)


Du coup après on choisit les regresseurs qu'on veut pour "av" (1) et "ap" (2) :

In [13]:
regresseurs1 = Classicreg + Times1reg + Xreg +Microreg + Macroreg

regresseurs2 = Classicreg + Times2reg + Xreg +  Microreg +  Macroreg

Regressions pour 2013-2019 :

In [39]:

#retrait des valeurs manquantes
BF4 =BF.where(BF['date'].astype(str)<'2020-01-01').dropna(subset=regresseurs1+['inflation anticipée'])


#du coup on re binarise Q25v2
BF4['anticipation baisse revenus'] = (BF4['anticipation baisse revenus']==3)*1
BF4['anticipation baisse revenus']=BF4.fillna(0)['anticipation baisse revenus']


BF4.dropna(subset=['weight'], inplace=True)

#transformation des colonnes en float
for element in regresseurs1 :
    BF4[element] = BF4[element].astype(float)

poids = BF4['weight'].astype('float')


Y1 = BF4['inflation anticipée'].astype(float)

X1 = BF4[regresseurs1] 

X1 = sm.add_constant(X1)
model1 = sm.WLS(Y1,X1, weights=poids)
results1 = model1.fit()
su1 = results1.summary()
print(su1)

                             WLS Regression Results                            
Dep. Variable:     inflation anticipée   R-squared:                       0.057
Model:                             WLS   Adj. R-squared:                  0.057
Method:                  Least Squares   F-statistic:                     231.7
Date:                 Tue, 25 Apr 2023   Prob (F-statistic):               0.00
Time:                         12:32:19   Log-Likelihood:            -3.0866e+05
No. Observations:                96101   AIC:                         6.174e+05
Df Residuals:                    96075   BIC:                         6.176e+05
Df Model:                           25                                         
Covariance Type:             nonrobust                                         
                                         coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------

Regressions pour été 2020-2022 :

In [15]:

#retrait des valeurs manquantes
BF5 =BF.where(BF['date'].astype(str)>'2020-06-20').dropna(subset=regresseurs2+['inflation anticipée'])


#du coup on re binarise Q25v2
BF5['anticipation baisse revenus'] = (BF5['anticipation baisse revenus']==3)*1
BF5['anticipation baisse revenus']=BF5.fillna(0)['anticipation baisse revenus']


BF5.dropna(subset=['weight'], inplace=True)

#transformation des colonnes en float
for element in regresseurs2 :
    BF5[element] = BF5[element].astype(float)

poids = BF5['weight'].astype('float')


Y2 = BF5['inflation anticipée'].astype(float)
X2 = BF5[regresseurs2]
X2 = sm.add_constant(X2)
model2 = sm.WLS(Y2,X2, weights=poids)
results2 = model2.fit()
su2 = results2.summary()
print(su2)

                             WLS Regression Results                            
Dep. Variable:     inflation anticipée   R-squared:                       0.103
Model:                             WLS   Adj. R-squared:                  0.102
Method:                  Least Squares   F-statistic:                     179.5
Date:                 Tue, 25 Apr 2023   Prob (F-statistic):               0.00
Time:                         12:15:35   Log-Likelihood:                -87796.
No. Observations:                26669   AIC:                         1.756e+05
Df Residuals:                    26651   BIC:                         1.758e+05
Df Model:                           17                                         
Covariance Type:             nonrobust                                         
                                         coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------

OK du coup là on finit pour la première régression

# Deuxième spécification

In [16]:
#J'ai pas touché à ça

list(BF.head())


"""
On doit définir les intervalles qui vont nous intéresser ici :

pour l'inflation à court terme: Q9 inflation supérieur à 4% (bin 1, bin 2 et bin 3)

déflation : bin6 à bin 10

long terme: Q9c+les mêmes bin

N.B : On considère pour la suite que les anticipations non remplies sont de 10 ( équiprobabilit de chaque tranche )
"""

BF['proba_st_high']=(BF['Q9_bin1'].fillna(10)+BF['Q9_bin2'].fillna(10)+BF['Q9_bin3'].fillna(10))

BF['proba_st_defl']=(BF['Q9_bin6'].fillna(10)+BF['Q9_bin7'].fillna(10)+BF['Q9_bin8'].fillna(10)+BF['Q9_bin9'].fillna(10)+BF['Q9_bin10'].fillna(10))

BF['proba_lt_high']=(BF['Q9c_bin1'].fillna(10)+BF['Q9c_bin2'].fillna(10)+BF['Q9c_bin3'].fillna(10))

BF['proba_lt_defl']=(BF['Q9c_bin6'].fillna(10)+BF['Q9c_bin7'].fillna(10)+BF['Q9c_bin8'].fillna(10)+BF['Q9c_bin9'].fillna(10)+BF['Q9c_bin10'].fillna(10))

Du coup le choix des regresseurs :

In [17]:

Classicreg = ['taux inflation']
Xreg = ['femme','age > 60 ans','age < 40 ans','Niveau licence','revenus > $100k','revenus < $50k','High_num']
XregProd = produitisation(BF,Xreg)

Timesreg = ['t'+str(y) for y in range(2013,2023)]


Macroreg = ['proba hausse chomage','proba hausse taux intêret','proba hausse taux intêret square','proba diminution indices boursiers']#des proba sur 100
MacroregProd = produitisation(BF,Macroreg)

Microreg = ['chômage','anticipation baisse revenus','proba chgt résidence','proba chgt résidence square','proba non-remboursement prêt']#indicatrice et des proba sur 100
MicroregPRod = produitisation(BF,Microreg)

#Wreg = ['PIB']
#WregProd = produitisation(BF,Wreg)

In [52]:
regresseurs1 = Classicreg  + Xreg  +Macroreg+Microreg


regresseurs2 = Classicreg + Times2reg + Xreg +  XregProd +  Macroreg




A - proba d'une inflation haute à court terme : 

Du coup je pensais après choisir à chaque fois la proba qu'on voulait expliquée mais on peut tout faire en même temps si vous préférez

In [19]:
regY = 'proba_st_high'
# regY = 'proba_st_defl'
# regY = 'proba_lt_high'
# regY = 'proba_lt_defl'

Et rebelote !

In [47]:
max(BF4[regY])

270.0

In [64]:
#retrait des valeurs manquantes
BF4 =BF.where(BF['date'].astype(str)<'2020-01-01').dropna(subset=regresseurs1+[regY])


#du coup on re binarise Q25v2
BF4['anticipation baisse revenus'] = (BF4['anticipation baisse revenus']==3)*1
BF4['anticipation baisse revenus']=BF4.fillna(0)['anticipation baisse revenus']


BF4.dropna(subset=['weight'], inplace=True)

#transformation des colonnes en float
for element in regresseurs1 :
    BF4[element] = BF4[element].astype(float)

poids = BF4['weight'].astype('float')



In [56]:

Y1 = BF4[regY]/100
X1 = BF4[regresseurs1].fillna(0).astype(float) #sur la periode 2013-2019 : [:97043]
X1 = sm.add_constant(X1)
model1 = sm.Logit(Y1,X1, weights=poids)
results1 = model1.fit()
su1 = results1.summary()
print(su1)


/usr/local/lib/python3.9/dist-packages/statsmodels/base/model.py:127: ValueWarning: unknown kwargs ['weights']
  warnings.warn(msg, ValueWarning)


Optimization terminated successfully.
         Current function value: 0.562125
         Iterations 5


/usr/local/lib/python3.9/dist-packages/statsmodels/base/model.py:127: ValueWarning: unknown kwargs ['weights']
  warnings.warn(msg, ValueWarning)


                           Logit Regression Results                           
Dep. Variable:          proba_st_high   No. Observations:                95970
Model:                          Logit   Df Residuals:                    95952
Method:                           MLE   Df Model:                           17
Date:                Tue, 25 Apr 2023   Pseudo R-squ.:                 0.01150
Time:                        12:40:49   Log-Likelihood:                -53947.
converged:                       True   LL-Null:                       -54575.
Covariance Type:            nonrobust   LLR p-value:                1.826e-256
                                         coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
const                                 -1.3445      0.046    -29.385      0.000      -1.434      -1.255
taux inflation                         0.0115      0.009   

In [48]:
BF4.drop(BF4[BF4[regY] > 100].index, inplace = True)

B- Inflation négative - court terme :

In [57]:
regY = 'proba_st_defl'
BF4.drop(BF4[BF4[regY] > 100].index, inplace = True)


In [58]:
Y1 = BF4[regY]/100
X1 = BF4[regresseurs1].fillna(0).astype(float) #sur la periode 2013-2019 : [:97043]
X1 = sm.add_constant(X1)
model1 = sm.Logit(Y1,X1, weights=poids)
results1 = model1.fit()
su1 = results1.summary()
print(su1)

/usr/local/lib/python3.9/dist-packages/statsmodels/base/model.py:127: ValueWarning: unknown kwargs ['weights']
  warnings.warn(msg, ValueWarning)


Optimization terminated successfully.
         Current function value: 0.283293
         Iterations 6


/usr/local/lib/python3.9/dist-packages/statsmodels/base/model.py:127: ValueWarning: unknown kwargs ['weights']
  warnings.warn(msg, ValueWarning)


                           Logit Regression Results                           
Dep. Variable:          proba_st_defl   No. Observations:                95811
Model:                          Logit   Df Residuals:                    95793
Method:                           MLE   Df Model:                           17
Date:                Tue, 25 Apr 2023   Pseudo R-squ.:                 -0.2058
Time:                        12:41:44   Log-Likelihood:                -27143.
converged:                       True   LL-Null:                       -22510.
Covariance Type:            nonrobust   LLR p-value:                     1.000
                                         coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
const                                 -0.9682      0.064    -15.153      0.000      -1.093      -0.843
taux inflation                        -0.0460      0.012   

C - Inflation élevée - court terme :

In [59]:
regY = 'proba_lt_high'
BF4.drop(BF4[BF4[regY] > 100].index, inplace = True)

In [60]:
Y1 = BF4[regY]/100
X1 = BF4[regresseurs1].fillna(0).astype(float) #sur la periode 2013-2019 : [:97043]
X1 = sm.add_constant(X1)
model1 = sm.Logit(Y1,X1, weights=poids)
results1 = model1.fit()
su1 = results1.summary()
print(su1)

/usr/local/lib/python3.9/dist-packages/statsmodels/base/model.py:127: ValueWarning: unknown kwargs ['weights']
  warnings.warn(msg, ValueWarning)


Optimization terminated successfully.
         Current function value: 0.588327
         Iterations 5


/usr/local/lib/python3.9/dist-packages/statsmodels/base/model.py:127: ValueWarning: unknown kwargs ['weights']
  warnings.warn(msg, ValueWarning)


                           Logit Regression Results                           
Dep. Variable:          proba_lt_high   No. Observations:                95775
Model:                          Logit   Df Residuals:                    95757
Method:                           MLE   Df Model:                           17
Date:                Tue, 25 Apr 2023   Pseudo R-squ.:                 0.01156
Time:                        12:43:18   Log-Likelihood:                -56347.
converged:                       True   LL-Null:                       -57006.
Covariance Type:            nonrobust   LLR p-value:                6.532e-270
                                         coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
const                                 -1.2695      0.045    -28.323      0.000      -1.357      -1.182
taux inflation                         0.0018      0.009   

D - Déflation - court terme :

In [61]:
regY = 'proba_lt_defl'
BF4.drop(BF4[BF4[regY] > 100].index, inplace = True)

In [62]:
Y1 = BF4[regY]/100
X1 = BF4[regresseurs1].fillna(0).astype(float) #sur la periode 2013-2019 : [:97043]
X1 = sm.add_constant(X1)
model1 = sm.Logit(Y1,X1, weights=poids)
results1 = model1.fit()
su1 = results1.summary()
print(su1)

/usr/local/lib/python3.9/dist-packages/statsmodels/base/model.py:127: ValueWarning: unknown kwargs ['weights']
  warnings.warn(msg, ValueWarning)


Optimization terminated successfully.
         Current function value: 0.321403
         Iterations 6


/usr/local/lib/python3.9/dist-packages/statsmodels/base/model.py:127: ValueWarning: unknown kwargs ['weights']
  warnings.warn(msg, ValueWarning)


                           Logit Regression Results                           
Dep. Variable:          proba_lt_defl   No. Observations:                95735
Model:                          Logit   Df Residuals:                    95717
Method:                           MLE   Df Model:                           17
Date:                Tue, 25 Apr 2023   Pseudo R-squ.:                 -0.1374
Time:                        12:45:44   Log-Likelihood:                -30770.
converged:                       True   LL-Null:                       -27053.
Covariance Type:            nonrobust   LLR p-value:                     1.000
                                         coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
const                                 -1.0770      0.060    -17.803      0.000      -1.196      -0.958
taux inflation                        -0.0097      0.012   

# Deuxième partie : sur les données post 2020 :

In [65]:
#retrait des valeurs manquantes
BF5 =BF.where(BF['date'].astype(str)>'2020-01-01').dropna(subset=regresseurs2+[regY])


#du coup on re binarise Q25v2
BF5['anticipation baisse revenus'] = (BF5['anticipation baisse revenus']==3)*1
BF5['anticipation baisse revenus']=BF5.fillna(0)['anticipation baisse revenus']


BF5.dropna(subset=['weight'], inplace=True)

#transformation des colonnes en float
for element in regresseurs2 :
    BF5[element] = BF5[element].astype(float)


                            WLS Regression Results                            
Dep. Variable:          proba_lt_defl   R-squared:                       0.049
Model:                            WLS   Adj. R-squared:                  0.048
Method:                 Least Squares   F-statistic:                     78.97
Date:                Tue, 25 Apr 2023   Prob (F-statistic):               0.00
Time:                        13:16:27   Log-Likelihood:            -1.5690e+05
No. Observations:               32333   AIC:                         3.138e+05
Df Residuals:                   32311   BIC:                         3.140e+05
Df Model:                          21                                         
Covariance Type:            nonrobust                                         
                                         coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
cons

A - Court terme - inflation élevée

In [68]:
regY = 'proba_st_high'
BF5.drop(BF5[BF5[regY] > 100].index, inplace = True)

In [69]:
Y1 = BF5[regY]/100
X1 = BF5[regresseurs1].fillna(0).astype(float) #sur la periode 2020-2022 
X1 = sm.add_constant(X1)
model1 = sm.Logit(Y1,X1, weights=poids)
results1 = model1.fit()
su1 = results1.summary()
print(su1)

/usr/local/lib/python3.9/dist-packages/statsmodels/base/model.py:127: ValueWarning: unknown kwargs ['weights']
  warnings.warn(msg, ValueWarning)
/usr/local/lib/python3.9/dist-packages/statsmodels/base/model.py:127: ValueWarning: unknown kwargs ['weights']
  warnings.warn(msg, ValueWarning)


Optimization terminated successfully.
         Current function value: 0.609797
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:          proba_st_high   No. Observations:                32298
Model:                          Logit   Df Residuals:                    32280
Method:                           MLE   Df Model:                           17
Date:                Tue, 25 Apr 2023   Pseudo R-squ.:                  0.1128
Time:                        13:18:58   Log-Likelihood:                -19695.
converged:                       True   LL-Null:                       -22199.
Covariance Type:            nonrobust   LLR p-value:                     0.000
                                         coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
const                                 -1.8044      0.074    

B - Court terme - déflation :

In [70]:
regY = 'proba_st_defl'
BF5.drop(BF5[BF5[regY] > 100].index, inplace = True)

In [71]:
Y1 = BF5[regY]/100
X1 = BF5[regresseurs1].fillna(0).astype(float) #sur la periode 2020-2022 
X1 = sm.add_constant(X1)
model1 = sm.Logit(Y1,X1, weights=poids)
results1 = model1.fit()
su1 = results1.summary()
print(su1)

/usr/local/lib/python3.9/dist-packages/statsmodels/base/model.py:127: ValueWarning: unknown kwargs ['weights']
  warnings.warn(msg, ValueWarning)
/usr/local/lib/python3.9/dist-packages/statsmodels/base/model.py:127: ValueWarning: unknown kwargs ['weights']
  warnings.warn(msg, ValueWarning)


Optimization terminated successfully.
         Current function value: 0.288386
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:          proba_st_defl   No. Observations:                32268
Model:                          Logit   Df Residuals:                    32250
Method:                           MLE   Df Model:                           17
Date:                Tue, 25 Apr 2023   Pseudo R-squ.:                 -0.1738
Time:                        13:19:56   Log-Likelihood:                -9305.6
converged:                       True   LL-Null:                       -7927.7
Covariance Type:            nonrobust   LLR p-value:                     1.000
                                         coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
const                                 -0.9744      0.106    

C - Long terme - Inflation haute :

In [72]:
regY = 'proba_lt_high'
BF5.drop(BF5[BF5[regY] > 100].index, inplace = True)

In [73]:
Y1 = BF5[regY]/100
X1 = BF5[regresseurs1].fillna(0).astype(float) #sur la periode 2020-2022 
X1 = sm.add_constant(X1)
model1 = sm.Logit(Y1,X1, weights=poids)
results1 = model1.fit()
su1 = results1.summary()
print(su1)

/usr/local/lib/python3.9/dist-packages/statsmodels/base/model.py:127: ValueWarning: unknown kwargs ['weights']
  warnings.warn(msg, ValueWarning)
/usr/local/lib/python3.9/dist-packages/statsmodels/base/model.py:127: ValueWarning: unknown kwargs ['weights']
  warnings.warn(msg, ValueWarning)


Optimization terminated successfully.
         Current function value: 0.617909
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:          proba_lt_high   No. Observations:                32253
Model:                          Logit   Df Residuals:                    32235
Method:                           MLE   Df Model:                           17
Date:                Tue, 25 Apr 2023   Pseudo R-squ.:                 0.04204
Time:                        13:21:54   Log-Likelihood:                -19929.
converged:                       True   LL-Null:                       -20804.
Covariance Type:            nonrobust   LLR p-value:                     0.000
                                         coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
const                                 -1.5266      0.073    

D - Moyen terme - Déflation :

In [74]:
regY = 'proba_lt_high'
BF5.drop(BF5[BF5[regY] > 100].index, inplace = True)

In [75]:
Y1 = BF5[regY]/100
X1 = BF5[regresseurs1].fillna(0).astype(float) #sur la periode 2020-2022 
X1 = sm.add_constant(X1)
model1 = sm.Logit(Y1,X1, weights=poids)
results1 = model1.fit()
su1 = results1.summary()
print(su1)

/usr/local/lib/python3.9/dist-packages/statsmodels/base/model.py:127: ValueWarning: unknown kwargs ['weights']
  warnings.warn(msg, ValueWarning)
/usr/local/lib/python3.9/dist-packages/statsmodels/base/model.py:127: ValueWarning: unknown kwargs ['weights']
  warnings.warn(msg, ValueWarning)


Optimization terminated successfully.
         Current function value: 0.617909
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:          proba_lt_high   No. Observations:                32253
Model:                          Logit   Df Residuals:                    32235
Method:                           MLE   Df Model:                           17
Date:                Tue, 25 Apr 2023   Pseudo R-squ.:                 0.04204
Time:                        13:23:02   Log-Likelihood:                -19929.
converged:                       True   LL-Null:                       -20804.
Covariance Type:            nonrobust   LLR p-value:                     0.000
                                         coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
const                                 -1.5266      0.073    